In [20]:
import networkx as nx
import pickle
import numpy as np
import torch
import torch.nn.functional as F
import copy

In [21]:
        path = './ia-conact/'
        with open(path+'train_edges','rb') as f:
            train_edges = pickle.load(f)
        with open(path+'train_neg_edges','rb') as f:
            train_neg_edges = pickle.load(f)
        with open(path+'pos_edges','rb') as f:
            pos_edges = pickle.load(f)
        with open(path+'neg_edges','rb') as f:
            neg_edges = pickle.load(f)
        with open(path+'feature_vec','rb') as f:
            feature_vec = pickle.load(f)

In [22]:
i=0
for key in feature_vec:
    isinf = np.isinf(feature_vec[key])
    if True in isinf:
        i+=1
print(i)

0


In [23]:
len(feature_vec)

274

In [24]:
def translate_data_into_graph(datapath,weight_idx,time_idx):
    edges=[]
    with open(datapath) as f:
        for line in f:
            tokens = line.strip().split()
            u = int(tokens[0])
            v = int(tokens[1])
            time = int(tokens[time_idx])
            
            if weight_idx:
                weight = int(tokens[weight_idx])
                edges.append((u,v,{'weight':weight,'time':time}))
            else:
                edges.append((u,v,{'time':time}))
            
    g = nx.MultiGraph()
    g.add_edges_from(edges)
    g = g.to_undirected()
    return g

In [25]:
datapath = './ia-conact/ia-contact.edges'
g = translate_data_into_graph(datapath,weight_idx=2,time_idx=3)

In [28]:
    def get_node(graph,time,first_node,second_node):
        
        g=graph
        
        cur_node = first_node
        first_node_neighbor = []
        for u,v,ddict in g.edges(cur_node,data=True):
            if ddict['time'] <time:
                first_node_neighbor.append((v,(ddict['time']-time)/500))
            
        cur_node = second_node
        second_node_neighbor = []
        for u,v,ddict in g.edges(cur_node,data=True):
            if ddict['time']<time:
                second_node_neighbor.append((v,(ddict['time']-time)/500))
        
        first_weight,second_weight = time_weight(first_node_neighbor,second_node_neighbor)
#         print(time)
#         print('*****************')
#         print(first_weight)
#         print(first_node_neighbor)
#         print("*****************")
#         print(second_weight)
#         print(second_node_neighbor)
        
        return first_node_neighbor,first_weight,second_node_neighbor,second_weight
    
    
    def time_weight(first_node_neighbor,second_node_neighbor):
        
        first_weight_sum = float(0)
        first_weight = []
        for (node,time) in first_node_neighbor:
            first_weight_sum +=  float(np.exp(time))
            first_weight.append(float(np.exp(time)))
        first_weight = np.array(first_weight)
            
        second_weight_sum = float(0)
        second_weight = []
        for (node,time) in second_node_neighbor:
            second_weight_sum += float(np.exp(time))
            second_weight.append(float(np.exp(time)))
        second_weight = np.array(second_weight)
        return first_weight/first_weight_sum,second_weight/second_weight_sum

In [29]:
get_node(g,101620,42,99)

([(1, -49.522),
  (1, -33.04),
  (1, -48.83),
  (1, -46.16),
  (1, -33.59),
  (1, -23.778),
  (1, -23.036),
  (1, -21.306),
  (1, -20.842),
  (2, -47.118),
  (2, -42.378),
  (2, -40.968),
  (2, -40.036),
  (2, -27.766),
  (2, -48.278),
  (2, -46.832),
  (2, -42.136),
  (2, -39.28),
  (2, -16.782),
  (3, -46.166),
  (3, -41.7),
  (3, -20.858),
  (3, -19.376),
  (3, -48.75),
  (3, -20.3),
  (3, -19.34),
  (3, -17.918),
  (3, -17.206),
  (4, -49.552),
  (4, -24.82),
  (4, -48.972),
  (4, -20.386),
  (4, -10.538),
  (5, -44.916),
  (5, -36.18),
  (5, -33.346),
  (5, -27.766),
  (5, -20.842),
  (5, -19.856),
  (5, -19.152),
  (5, -14.64),
  (5, -11.94),
  (5, -11.218),
  (5, -44.342),
  (5, -43.4),
  (5, -35.58),
  (5, -32.484),
  (5, -27.398),
  (5, -26.884),
  (5, -19.026),
  (5, -18.312),
  (5, -16.176),
  (5, -14.02),
  (5, -11.576),
  (5, -11.078),
  (6, -40.73),
  (6, -27.518),
  (6, -26.526),
  (6, -24.82),
  (6, -26.692),
  (6, -22.912),
  (7, -49.552),
  (7, -47.836),
  (7, -46.878

In [30]:
train_features = train_edges+train_neg_edges
test_feature = pos_edges+neg_edges

In [31]:
import random
random.shuffle(train_features)
random.shuffle(test_feature)

In [32]:
        processed_train_feature=[]
        for u,v,ddict in train_features:
            if feature_vec.get(u) is None or feature_vec.get(v) is None:
                continue
            
            label = 0
            if ddict['time'] is None:
                label = 0.0
                first_node_neighbor,first_weight,second_node_neighbor,second_weight = get_node(g,0,u,v)
            else:
                label =1.0
                first_node_neighbor,first_weight,second_node_neighbor,second_weight = get_node(g,ddict['time'],u,v)
            
            
            
            first_weight = torch.tensor(first_weight,dtype=torch.float32)
            second_weight = torch.tensor(second_weight,dtype=torch.float32)
            
            
            h = copy.deepcopy(feature_vec.get(u))
            h = torch.from_numpy(h)
            first_node_feature = h
            for i in range(len(first_node_neighbor)):
                u = feature_vec.get(first_node_neighbor[i][0])
            
                if u is None:
                    continue
                u = copy.deepcopy(feature_vec.get(first_node_neighbor[i][0]))
                u = torch.from_numpy(u)
                first_node_feature+= u*first_weight[i]
            
            
            h = copy.deepcopy(feature_vec.get(v))
            h = torch.from_numpy(h)    
            second_node_feature = h
            for i in range(len(second_node_neighbor)):
                u = feature_vec.get(second_node_neighbor[i][0])
            
                if u is None:
                    continue
                u = copy.deepcopy(feature_vec.get(second_node_neighbor[i][0]))
                u = torch.from_numpy(u)
                second_node_feature += u*second_weight[i]
            
#             first_node_feature = first_node_feature.view(-1,1000)
#             first_node_feature = F.normalize(first_node_feature,p=2,dim=1)
#             second_node_feature = second_node_feature.view(-1,1000)
#             second_node_feature = F.normalize(first_node_feature,p=2,dim=1)
            feature = torch.cat((first_node_feature,second_node_feature),0)
            
            feature = feature.view(-1,2000)
            
            processed_train_feature.append((feature,label))

In [33]:
len(processed_train_feature)

44816

In [34]:
with open('./ia-contact-features/train_cat_features_v1','wb') as f:
    pickle.dump(processed_train_feature, f)

In [35]:
    processed_test_feature = []
    for u,v,ddict in test_feature:
        if feature_vec.get(u) is None or feature_vec.get(v) is None:
            continue
        
        label = 0
        if ddict['time'] is None:
            label = 0
            first_node_neighbor,first_weight,second_node_neighbor,second_weight = get_node(g,0,u,v)
        else:
            label = 1
            first_node_neighbor,first_weight,second_node_neighbor,second_weight = get_node(g,ddict['time'],u,v)
                
        first_weight = torch.tensor(first_weight,dtype=torch.float32)
        second_weight = torch.tensor(second_weight,dtype=torch.float32)
         
        
        h = copy.deepcopy(feature_vec.get(u))
        h = torch.from_numpy(h)
        first_node_feature = h
        for i in range(len(first_node_neighbor)):
            u = feature_vec.get(first_node_neighbor[i][0])
            
            if u is None:
                continue
            u = copy.deepcopy(feature_vec.get(first_node_neighbor[i][0]))
            u = torch.from_numpy(u)
            first_node_feature+= u*first_weight[i]
            
        h = copy.deepcopy(feature_vec.get(v))
        h = torch.from_numpy(h)
        second_node_feature = h
        for i in range(len(second_node_neighbor)):
            u = feature_vec.get(second_node_neighbor[i][0])
            
            if u is None:
                continue
            u = copy.deepcopy(feature_vec.get(second_node_neighbor[i][0]))
            u = torch.from_numpy(u)
            second_node_feature += u*second_weight[i]
        
#         first_node_feature = first_node_feature.view(-1,1000)
#         first_node_feature = F.normalize(first_node_feature,p=2,dim=1)
#         second_node_feature = second_node_feature.view(-1,1000)
#         second_node_feature = F.normalize(first_node_feature,p=2,dim=1)
        
        feature = torch.cat((first_node_feature,second_node_feature),0)
        
        feature = feature.view(-1,2000)
        processed_test_feature.append((feature,label))

In [36]:
with open('./ia-contact-features/test_cat_features_v1','wb') as f:
    pickle.dump(processed_test_feature, f)

In [41]:
processed_train_feature[0]

(tensor([[-0.0621, -1.3374,  2.6228,  ..., -0.0722,  2.3771,  0.9484]],
        dtype=torch.float64),
 1.0)

In [38]:
processed_test_feature[0]

(tensor([[-1.1809,  1.2124,  1.2864,  ...,  0.4239,  2.0086,  1.2304]],
        dtype=torch.float64),
 1)

In [39]:
res = 0
for f,label in processed_train_feature:
    if label ==0:
        res+=1
print(res/len(processed_train_feature))

0.5


In [40]:
res = 0
for f,label in processed_test_feature:
    if label ==0:
        res+=1
print(res/len(processed_test_feature))

0.5
